In [1]:
library(R6)

In [2]:
MDMM <- R6Class("MDMM",
  # public methods and parameters
  public = list(
  initialize = function(){},
      
  fit = function(Xs,Ks,intial.alpha=NULL, max.iter=200, threshold=0.001, learning.rate=0.1){
      
      ## input
      ## Xs: group of count vectors
      ## Ks: group of mask vectors
      ## Initial_alpha: the initial value of alpha, can be a vector, the default value is the mean value of all Xs
      
      ###### parameter validation ######
      if (!sum(dim(Xs)==dim(Ks))==2){
          print("Error! The dimension of input Xs and Ks must be same.")
          return(-1)
      }
      if (!is.null(intial.alpha) && !dim(Xs)[2]==length(intial.alpha)){
          print("Error! The column dimension of input Xs and the length of intial.alpha must be same.")
          return(-1)
      }
      
      ###### parameter initialize ######
      private$dims = dim(Xs)[2]
      private$learning_rate <- learning.rate
      private$max_iter <- max.iter
      private$threshold <- threshold
      
      if(is.null(intial.alpha)){
          initial.alpha <- colMeans(Xs)
      }
      private$initial_alpha <- initial.alpha
      alpha <- initial.alpha
    
      private$LL_list <- c(private$log.Likelihood(alpha,Xs,Ks))
      
      ###### estimation ######
      for(epoch in 1:private$max_iter){
        for(sample in 1:dim(Xs)[1]){
            gradient.alpha <- matrix(0, private$dims, 1)
            K <- Ks[sample,]
            X <- Xs[sample,]

            for(j in 1: private$dims){
                if(K[j]==1){
                    gradient.alpha[j] <- digamma(sum(alpha))-digamma(sum(alpha)+sum(X))+digamma(X[j]+alpha[j])-digamma(alpha[j])
                }
            }
            alpha <- alpha + private$learning_rate * as.double(gradient.alpha)
        }
        if(abs(private$LL_list[length(private$LL_list)]-private$log.Likelihood(alpha,Xs,Ks))<= private$threshold){
            break
        }else{
            private$LL_list <- c(private$LL_list, private$log.Likelihood(alpha,Xs,Ks))
        }  
    }
    
      ###### confidence range ######
      high_boundary <- matrix(1,1,private$dims)
      low_boundary <- matrix(1,1,private$dims)
  
      for(i in 1: private$dims){
          Second.derivative.i <- 0
          for(sample in 1:dim(Xs)[1]){
              K <- Ks[sample,]
              X <- Xs[sample,]
              Second.derivative.i <- Second.derivative.i + trigamma(sum(alpha))-trigamma(sum(alpha)+sum(X))+trigamma(X[i]+alpha[i])-trigamma(alpha[i])
          }
              high_boundary[i] <- alpha[i] + 1.96/sqrt(-1*Second.derivative.i)
              low_boundary[i] <- max(alpha[i] - 1.96/sqrt(-1*Second.derivative.i),0)
      }
      
      
      ###### save result ######
      result <- as.data.frame(rbind(as.double(high_boundary),alpha,as.double(low_boundary)))
      row.names(result)<- c("high_boundary","estimated_alpha","low_boundary")
      private$estimation <- result 
  },
      
      
  get_parameters = function(){
      print(paste("dims:",private$dims))
      print(paste("learning_rate:",private$learning_rate))
      print(paste("max_iter:",private$max_iter))
      print(paste("threshold:",private$threshold))
  },
  get_result = function(){return(private$estimation)},
  draw_trainning_process = function(){plot(private$LL_list,xlab =  "epoch", ylab ="Log.Likelihood")}
  ), 
                
  
  private = list(
      ## private parameters
      dims = NULL,
      learning_rate = NULL,
      initial_alpha = NULL,
      threshold = NULL,
      max_iter = NULL,
      
      estimation = NULL,
      LL_list = NULL,
      
      ## private methods
      log.Likelihood = function(alpha,Xs,Ks){
        Likelihood <- 0
        for(i in 1:dim(Xs)[1]){
            K_i <- Ks[i,]
            X_i <- as.double(Xs[i,K_i==1])
            alpha_i <- as.double(alpha[K_i==1])

            Likelihood <- Likelihood + log(gamma(sum(alpha_i))) - log(gamma(sum(alpha_i+X_i)))
            for(j in 1:length(alpha_i)){
                Likelihood <- Likelihood + log(gamma(alpha_i[j] + X_i[j])/gamma(alpha_i[j]))
            }
        }
        return(Likelihood)
    }
  )
)

# load data

In [7]:
df_proportion <- read.csv(file = "Cell_proportion_fetal.csv")

In [8]:
df_proportion

X,donor_id,Age,subregion,Endothelial.cell,Cardiomyocyte.cell,Myelocyte,Fibroblast,Lymphocyte,Smooth.muscle.cell,Neuron,Mesothelial.cell,Epithelial.cell,Pericyte,Adipocyte
<int>,<chr>,<dbl>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
3,HE6W_1,6.0,Left atria,10,11,4,0,0,8,0,1,0,0,0
4,HE6W_1,6.0,Left ventricle,16,7,2,0,0,16,0,0,0,0,0
5,HE6W_1,6.0,Right atria,10,16,0,0,0,8,10,1,0,0,0
6,HE6W_1,6.0,Right ventricle,6,13,0,0,0,24,0,0,0,0,0
7,HE6W_2,6.0,Left atria,15,3,5,0,1,19,0,4,0,0,0
8,HE6W_2,6.0,Left ventricle,13,9,2,0,0,23,0,0,0,0,0
9,HE6W_2,6.0,Right atria,15,4,3,0,0,23,0,2,0,0,0
10,HE6W_2,6.0,Right ventricle,17,5,0,0,0,26,0,0,0,0,0
86,Asp_donor1,6.5,NA,804,762,75,613,0,706,75,0,137,302,0


In [9]:
unique(df_proportion$Age)

[1]  6.0  6.5  7.0  9.0 10.0 11.0 12.0 13.0 13.5 15.0 16.0 16.5 17.0 17.5 20.0
[16] 22.0 23.0 24.0 25.0